In [1]:
import numpy as np
import pandas as pd

from xgboost import XGBRegressor


from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as MAE
from math import sqrt

/home/thomas/Downloads/venv/lib/python3.5/site-packages/xgboost/__init__.py:29: FutureWarning: Python 3.5 support is deprecated; XGBoost will require Python 3.6+ in the near future. Consider upgrading to Python 3.6+.
  FutureWarning)


In [2]:
train_raw = pd.read_csv("train_raw.csv")
test_raw = pd.read_csv("test_raw.csv")


X_train = train_raw.loc[:, train_raw.columns != 'target']
Y_train = train_raw.loc[:, train_raw.columns == 'target']

X_test = test_raw.loc[:, test_raw.columns != 'target']
Y_test = test_raw.loc[:, test_raw.columns == 'target']

In [3]:
X_train

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.07151,0.0,4.49,0.0,0.4490,6.121,56.8,3.7476,3.0,247.0,18.5,395.15,8.44
1,0.14231,0.0,10.01,0.0,0.5470,6.254,84.2,2.2565,6.0,432.0,17.8,388.74,10.45
2,3.84970,0.0,18.10,1.0,0.7700,6.395,91.0,2.5052,24.0,666.0,20.2,391.34,13.27
3,6.44405,0.0,18.10,0.0,0.5840,6.425,74.8,2.2004,24.0,666.0,20.2,97.95,12.03
4,19.60910,0.0,18.10,0.0,0.6710,7.313,97.9,1.3163,24.0,666.0,20.2,396.90,13.44
5,0.04011,80.0,1.52,0.0,0.4040,7.287,34.1,7.3090,2.0,329.0,12.6,396.90,4.08
6,1.80028,0.0,19.58,0.0,0.6050,5.877,79.2,2.4259,5.0,403.0,14.7,227.61,12.14
7,0.13960,0.0,8.56,0.0,0.5200,6.167,90.0,2.4210,5.0,384.0,20.9,392.69,12.33
8,11.08740,0.0,18.10,0.0,0.7180,6.411,100.0,1.8589,24.0,666.0,20.2,318.75,15.02
9,0.23912,0.0,9.69,0.0,0.5850,6.019,65.3,2.4091,6.0,391.0,19.2,396.90,12.92


In [4]:
model = XGBRegressor()
model.fit(X_train,Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [5]:
Y_test_predicted = model.predict(X_test)
Y_test["target_predicted"] = Y_test_predicted

/home/thomas/Downloads/venv/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
mae_result = MAE(Y_test["target"], Y_test["target_predicted"])
rms_result = sqrt(mae_result)
r2_score_result = r2_score(Y_test["target"], Y_test["target_predicted"])

print("mae_result : ", mae_result)
print("rms_result : ", rms_result)
print("r2_score_result : ", r2_score_result)

mae_result :  2.4991093086568927
rms_result :  1.580857143658747
r2_score_result :  0.8440950019402467


In [7]:
Y_test.head()

,target,target_predicted
0,37.6,49.420609
1,27.9,26.236307
2,22.6,23.059195
3,13.8,20.176821
4,35.2,39.261818
